In [1]:
import os
import filetype
from shutil import copyfile, move
import pandas as pd
import pigeonXT as pixt
# import utils.colombe as pixt
from ipywidgets import IntProgress, HBox, Label
from IPython.display import clear_output, display, Image as IPythonImage
from utils.pdf import pdfToImageFiles
from pathlib import Path

In [2]:
labels =['fr-id1995-recto', 'fr-id1995-verso', 'fr-id2021-recto', 'fr-id2021-verso', 'fr-passport', 'fr-rib', 'fr-kbis', 'invoice', 'document']

source = 'data/'
new = source + 'new/'
if not os.path.exists(source + 'train'):
    os.makedirs(source + 'train')
csv = 'data/train.csv'
train = source + 'train/'
if not os.path.exists(source + 'archive'):
    os.makedirs(source + 'archive')
archive = source + 'archive/'
if not os.path.exists(source + 'error'):
    os.makedirs(source + 'error')
error = source + 'error/'

trainDf = pd.DataFrame(columns=['path', 'labels'])
if not os.path.exists(source + 'train.csv'):
    trainDf.to_csv(csv, mode='a', index=False, header=True)


In [3]:
def saveFile(path, labels):
    move(path, train + Path(path).name)
    df = pd.DataFrame([[Path(path).name, ' '.join(labels)]])
    df.to_csv(csv, mode='a', index=False, header=False)

def prepare ():
    files = [f for f in os.listdir(new) if os.path.isfile(new + f)]
    for file in files:
        kind = filetype.guess(new + file)
    
        if kind is None:
            move(new + file, error + file)
            continue

        if 'application/pdf' == kind.mime:
            pdfToImageFiles(new + file, new)
            move(new + file, archive + file)

def listdir_fullpath(d):
    return ['./' + os.path.join(d, f) for f in os.listdir(d)]

def processFiles():
    annotations = pixt.annotate(
        listdir_fullpath(new), 
        options=labels, 
        task_type='multilabel-classification',
        buttons_in_a_row=5,
        reset_buttons_after_click=True,
        include_next=True,
        include_back=True,
        example_column='path',
        display_fn=lambda filename: display(IPythonImage(filename, width = 400, height = 400)),
        example_process_fn=saveFile
    )

prepare()
processFiles()

HTML(value='0 of 108 Examples annotated, Current Position: 0 ')

Output()